In [2]:
import io
import os
import re
import sys
import csv
import gzip
import glob
import time
import secrets
import multiprocessing
import subprocess
import resource
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from plotnine import *
import scipy as sp
import statsmodels.api as sm
import random
from collections import Counter
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
import itertools
import collections
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *

from scipy.stats import poisson
from scipy.stats import chi2
from scipy.stats import friedmanchisquare
from scipy.stats import studentized_range
pd.options.mode.chained_assignment = None
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

%cd /gpfs3/well/band/users/rbx225/GAMCC/

# from matplotlib import colormaps
# list(colormaps)

/gpfs3/well/band/users/rbx225/GAMCC


In [17]:
# mini = False
# common_cols = ['chr', 'pos', 'ref', 'alt']

# imdir = "results/imputation_comparison/vanilla/lc_oneKG_hrc--hc--snps_gw/"

# quilt_vcf = imdir + "vcf/all_samples/lc_vcf/lc.subset.chr19.vcf.gz"
# chip_vcf = imdir + "vcf/all_samples/hc_vcf/hc.chr19.vcf.gz"
# af_txt = "data/gnomAD_MAFs/afr/gnomAD_MAF_afr_chr19.txt"

# chip, lc, af = lcwgsus.imputation_calculation_preprocess(chip_vcf, quilt_vcf, af_txt)

h_report = lcwgsus.calculate_h_imputation_accuracy(chip, lc, af)
h_report = h_report.drop(columns = common_cols)

h_impacc = lcwgsus.generate_h_impacc(h_report)

v_report = lcwgsus.calculate_v_imputation_accuracy(chip, lc, af)

v_impacc = lcwgsus.generate_v_impacc(v_report)

In [2]:
panel = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/20181129_HLA_types_full_1000_Genomes_Project_panel.txt', sep = '\t')
panel

,Region,Population,Sample ID,HLA-A 1,HLA-A 2,HLA-B 1,HLA-B 2,HLA-C 1,HLA-C 2,HLA-DQB1 1,HLA-DQB1 2,HLA-DRB1 1,HLA-DRB1 2
0,AFR,ACB,HG01879,23:01,68:02,13:02,42:01,08:04,17:01,02:02,04:02,03:02,09:01
1,AFR,ACB,HG01880,33:03,68:02,40:06,42:01,12:02,17:01,02:01,03:04,03:01,11:06
2,AFR,ACB,HG01882,23:01,34:02,07:02,44:50,04:01,07:02,06:02,06:02,15:03,15:03
3,AFR,ACB,HG01883,02:01,68:02,53:01,44:03,04:01,04:01,02:02,06:02,13:03,15:03
4,AFR,ACB,HG01885,03:01,33:03,35:01,35:01,04:01,07:18,02:02,03:19,13:04,07:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,SAS,STU,HG04114,02:01,24:02,52:01,55:01,01:02,12:02,03:02,06:03,13:01,04:03
2689,SAS,STU,HG04127,02:11,24:02,15:05,52:01,03:03,12:02,06:09,06:09,13:01,13:02
2690,SAS,STU,HG04210,02:01,24:02,15:25,35:01,04:01,07:26,05:02,06:03,13:01,14:02
2691,SAS,STU,HG04227,01:01,01:01,57:01,44:03,06:02,07:06,03:03,06:01,07:01,15:01


In [13]:
afr = panel[panel['Region'] == 'AFR']
afr = afr[afr['Population'] == 'GWD']
set(afr['HLA-DRB1 1'].unique())
# afr

{'01:01',
 '01:02',
 '03:01',
 '03:02',
 '04:05',
 '04:06',
 '07:01',
 '08:04',
 '08:06',
 '09:01',
 '10:01',
 '11:01',
 '11:02',
 '11:04',
 '13:01',
 '13:02',
 '13:03',
 '13:04',
 '15:03'}

In [6]:
hla = lcwgsus.read_hla_direct_sequencing()
hla_alleles_df = lcwgsus.gather_hla_alleles(hla)
drb1 = hla_alleles_df[hla_alleles_df['Locus'] == 'DRB1']
drb1

,Locus,Allele
4,DRB1,DRB1*09:01/09:31
9,DRB1,DRB1*01:02
14,DRB1,DRB1*13:04
19,DRB1,DRB1*11:01
24,DRB1,DRB1*13:04
...,...,...
2479,DRB1,DRB1*09:01/09:31
2484,DRB1,DRB1*11:01
2489,DRB1,DRB1*03:02
2494,DRB1,DRB1*13:02


In [7]:
drb1['Allele'].str.split('*').str.get(1).unique()

array(['09:01/09:31', '01:02', '13:04', '11:01', '07:139/07:01',
       '09:21/09:01/09:31', '07:139/07:01/07:79/07:93', '03:02', '10:01',
       '13:03', '11:02', '08:06', '04:05', '03:01', '08:04', '13:01',
       '01:83/01:02/01:136', '13:02', '04:354/04:06', '01:136', '-9',
       '04:01', '04:03', '03:147/03:01/03:183/03:199', '16:02',
       '13:327/13:01/13:313', '15:185/15:178/15:03', '15:185/15:03',
       '14:243/14:54', '10:38Q/10:01', '12:10/12:01'], dtype=object)

{'01:01',
 '01:02',
 '01:03',
 '01:23*',
 '03:01',
 '03:02',
 '04:01',
 '04:03',
 '04:04',
 '04:05',
 '04:06',
 '04:07',
 '07:01',
 '08:02',
 '08:04',
 '08:06',
 '09:01',
 '10:01',
 '11:01',
 '11:02',
 '11:04',
 '11:10',
 '12:01',
 '13:01',
 '13:02',
 '13:02*',
 '13:03',
 '13:04',
 '13:177*',
 '14:54',
 '15:01',
 '15:03'}

# HLA

In [2]:
chip = lcwgsus.read_hla_chip_imputation_results('/well/band/users/rbx225/GAMCC/results/hla/server/chip_vanilla/chr6.dose.vcf.gz')
chip

,SampleID,Locus,One field1,Two field1,One field2,Two field2
0,GAM013489,A,02,02:01,02,02:01
1,GAM013489,B,44,44:03,82,82:01
2,GAM013489,C,03,03:02,03,03:03
3,GAM013489,DQB1,02,02:01,05,05:02
4,GAM013489,DRB1,09,09:01,16,16:02
...,...,...,...,...,...,...
925,GAM975418,A,29,29:02,33,33:01
926,GAM975418,B,07,07:02,07,07:02
927,GAM975418,C,15,15:05,15,15:05
928,GAM975418,DQB1,04,04:02,05,05:01


In [3]:
x = lcwgsus.read_hla_lc_imputation_results('/well/band/users/rbx225/GAMCC/results/hla/imputation/batches/')
# x = x[x['Locus'] == 'DRB1'].sort_values(by = ['SampleID', 'Locus']).reset_index(drop = True)
x

,SampleID,Locus,One field1,Two field1,One field2,Two field2,prob
0,GAM013489,A,02,02:01,02,02:01,0.958238
1,GAM013489,B,44,44:03,82,82:01,0.999901
2,GAM013489,C,03,03:02,03,03:03,0.999901
3,GAM013489,DQB1,02,02:02,05,05:02,1.000000
4,GAM013489,DRB1,07,07:01,16,16:02,0.986893
...,...,...,...,...,...,...,...
1045,GAM975418,A,29,29:02,33,33:03,0.655144
1046,GAM975418,B,07,07:02,07,07:02,0.999995
1047,GAM975418,C,15,15:05,15,15:05,1.000000
1048,GAM975418,DQB1,04,04:02,05,05:01,1.000000


In [6]:
compared = lcwgsus.compare_hla_types(x, chip)
compared
hla_report = lcwgsus.generate_hla_imputation_report(compared, 'm')

In [7]:
hla_report

,Locus,Concordance,Resolution,Source
0,A,0.895161,One field,m
1,A,0.803763,Two field,m
2,B,0.930108,One field,m
3,B,0.881720,Two field,m
4,C,0.975806,One field,m
5,C,0.895161,Two field,m
6,DQB1,0.989247,One field,m
7,DQB1,0.521505,Two field,m
8,DRB1,0.672043,One field,m
9,DRB1,0.553763,Two field,m
